compute_regional_employment.py
Jay Sayre - sayrejay@gmail.com

Computes estimated share of employment in each municipality of the Dominican Republic in a given ISIC (International Standard Industrial Classification) code using data provided by IPUMS international at the ISIC 2-digit level on workers' occupations and data provided by the DR's ONE (Oficina Nacional de Estadística) at the ISIC 4-digit level on companies (empresas) registered in the D.R.

Inputs:
DirectoryofCompaniesandEstablishments/data/mergedprov.csv - D.R. provided information on the companies (and their sizes and industrial classifications) in each municipality. Merged together from separate files in "data/" folder using a UNIX one-liner.

Intermediate data:
DirectoryofCompaniesandEstablishments/companiesandsizesbymunicipality.csv - Clean and aggegrate input companies data (mergedprov.csv) to the municipality level, computing number of companies of a given size in a given industry

DirectoryofCompaniesandEstablishments/municipalitycodecorrespondence.csv - Takes input data, strips data of ll information except for geocodes and corresponding province/municipality names

Outputs:




In [7]:
import pandas as pd
import os

if os.name == 'nt':
    tdir ="D:/Dropbox/Dropbox (Personal)/College/DR_Paper/"
else:
    tdir ="/home/j/Dropbox/College/DR_Paper/"
sndlvl = "DirectoryofCompaniesandEstablishments/"
    
## Inputs
empresadata = tdir+sndlvl+"data/mergedprov.csv"
occu2002 = tdir+'occupationbymunicipality2002.csv'
occu2010 = tdir+'occupationbymunicipality2010.csv'
## Outputs
muncodecorres = tdir+sndlvl+'municipalitycodecorrespondence.csv'
companiesbysize = tdir+sndlvl+'companiesandsizesbymunicipality.csv'

In [32]:
## Create correspondence table of municis to codes found in Empresa data
## Only need to run this once
'''
provlistdf = pd.read_csv(empresadata,encoding='utf-8')
provlistdf['ISIC'] = provlistdf['CLASS'].apply(lambda x: "'"+x[-4:])
provdrops = [u'SECT',u'DIV',u'GROUP',u'CLASS',u'SECT_DESC',
            u'DIV_DESC',u'GROUP_DESC',u'CLASS_DESC',u'GEOCODE',
            u'PROVINCE_DESC', u'MUNICIPIO_DESC']
provdrops.remove(u'PROVINCE_DESC')
provdrops.remove(u'MUNICIPIO_DESC')
provdrops.extend(['ISIC'])
provlistdf = provlistdf.drop(provdrops, 1)
provlistdf = provlistdf.groupby(['PROVINCE','MUNICIPIO','PROVINCE_DESC', 'MUNICIPIO_DESC'], as_index=False).sum()
provlistdf = provlistdf.drop('EMPRESAS', 1)

## Commented out so it won't write to file everytime
#provlistdf.to_csv(muncodecorres,index=False,encoding="utf-8")
'''

In [4]:
## Directory of Empresas
## Calculate share of companies in a industry of a given size in each mun

empresadf = pd.read_csv(empresadata,encoding='utf-8')
empresadf['ISIC'] = empresadf['CLASS'].apply(lambda x: "'"+x[-4:])
dropcols = [u'SECT',u'DIV',u'GROUP',u'CLASS',u'SECT_DESC',
            u'DIV_DESC',u'GROUP_DESC',u'CLASS_DESC',u'GEOCODE',
            u'PROVINCE_DESC', u'MUNICIPIO_DESC']

empresadf.drop(dropcols, 1, inplace=True)

## Weighting by company size
def csize(word):
    if word == u'050 a 099':
        return "50-99"
    elif word == u'010 a 049':
        return "10-49"
    elif word == u'001 a 009':
        return "1-9"
    elif word == u'250 o m\xe1s':
        return "250-"
    elif word == u'100 a 249':
        return "100-249"
    else:
        return word
empresadf['SIZE'] = empresadf['SIZE'].apply(lambda x: csize(x))

## Group Empresas by Municipality, ISIC Code, and size
empresadf = empresadf.groupby(['PROVINCE','MUNICIPIO','ISIC','SIZE'], as_index=False).sum()

## Calculate total number of companies by size in province and municipality
municidf = empresadf.groupby(['PROVINCE','MUNICIPIO','SIZE'], as_index=False).sum()
provdf = empresadf.groupby(['PROVINCE','SIZE'], as_index=False)['EMPRESAS'].sum()
municidf.columns = ['PROVINCE','MUNICIPIO','SIZE','MUNICINUM']
provdf.columns = ['PROVINCE','SIZE','PROVNUM']

empresadf = empresadf.merge(municidf, on=['PROVINCE','MUNICIPIO','SIZE'], how='left')
empresadf = empresadf.merge(provdf, on=['PROVINCE','SIZE'], how='left')

#empresadf.to_csv(companiesbysize,index=False)

In [9]:
## Calibrate above data with IPUMS industry data for 2002 and 2010
## IPUMS data is the output of IPUMSdataaggregation.py

occudf2002 = pd.read_csv(occu2002)
occudf2010 = pd.read_csv(occu2010)


,mun,1,11,12,13,21,22,23,24,31,...,share72,share73,share74,share81,share82,share83,share91,share92,share93,share98
0,101,281,18,248,435,616,548,712,1410,431,...,0.034338,0.008239,0.026157,0.000864,0.020914,0.027885,0.146569,0.000230,0.019243,0.012560
1,201,24,3,5,2,2,47,69,20,10,...,0.044824,0.002134,0.027748,0.003202,0.006403,0.057631,0.172892,0.024546,0.046958,0.016009
2,202,5,2,0,1,0,30,8,1,1,...,0.033557,0.000000,0.023490,0.000000,0.010067,0.026846,0.208054,0.077181,0.050336,0.016779
3,203,1,16,3,0,0,12,16,2,1,...,0.005495,0.000000,0.027473,0.000000,0.016484,0.043956,0.148352,0.021978,0.065934,0.016484
4,204,6,1,4,1,0,28,18,4,2,...,0.028409,0.002841,0.011364,0.002841,0.014205,0.051136,0.170455,0.042614,0.034091,0.022727
5,205,5,2,0,1,0,30,8,1,1,...,0.033557,0.000000,0.023490,0.000000,0.010067,0.026846,0.208054,0.077181,0.050336,0.016779
6,206,6,1,4,1,0,28,18,4,2,...,0.028409,0.002841,0.011364,0.002841,0.014205,0.051136,0.170455,0.042614,0.034091,0.022727
7,207,24,3,5,2,2,47,69,20,10,...,0.044824,0.002134,0.027748,0.003202,0.006403,0.057631,0.172892,0.024546,0.046958,0.016009
8,208,1,16,3,0,0,12,16,2,1,...,0.005495,0.000000,0.027473,0.000000,0.016484,0.043956,0.148352,0.021978,0.065934,0.016484
9,209,1,16,3,0,0,12,16,2,1,...,0.005495,0.000000,0.027473,0.000000,0.016484,0.043956,0.148352,0.021978,0.065934,0.016484
